In [1]:
%config Completer. use_jedi = False 
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import metpy.calc as mpcalc
from metpy.units import units
from datetime import datetime

##### Load Data Pre Processing

In [2]:
files = glob.glob('load/*.csv')

###### Checking if column names are same across data

In [3]:
cols = []
for file in files:
    cols.append(list(pd.read_csv(file).columns))

In [4]:
cols = set.intersection(*[set(x) for x in cols])


In [5]:
li = []
for file in files:
    dfLoad = pd.read_csv(file,index_col= None, header = 0, usecols=['RTD End Time Stamp', 'RTD Actual Load'])
    li.append(dfLoad)
dfLoad = pd.concat(li, axis = 0, ignore_index=True)

In [6]:
dfLoad = dfLoad.rename(columns= {'RTD End Time Stamp':'time_stamp', 'RTD Actual Load':'rtd_actual_load'})

In [7]:
dfLoad.time_stamp = pd.to_datetime(dfLoad.time_stamp).dt.strftime('%Y-%m-%d %H:%M')


In [8]:
def hour_sampling(df):
    df.time_stamp = pd.to_datetime(dfLoad.time_stamp)
    df = df.set_index('time_stamp').iloc[:-1,:].resample('H').mean()
    return df

In [9]:
dfLoad = hour_sampling(dfLoad)

In [10]:
dfLoad.to_csv('CleanLoadData.csv', index = False)

In [11]:
dfLoad

,rtd_actual_load
time_stamp,
2010-01-01 00:00:00,5385.227273
2010-01-01 01:00:00,5192.575000
2010-01-01 02:00:00,4978.225000
2010-01-01 03:00:00,4784.608333
2010-01-01 04:00:00,4696.741667
...,...
2021-05-03 09:00:00,5285.681958
2021-05-03 10:00:00,5342.884858
2021-05-03 11:00:00,5418.255667


In [12]:
files = glob.glob('weather/*.csv')

In [13]:
files[:5]

['weather\\2010.csv',
 'weather\\2011.csv',
 'weather\\2012.csv',
 'weather\\2013.csv',
 'weather\\2014.csv']

In [14]:
cols = []
for file in files:
    cols.append(list(pd.read_csv(file).columns))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,22,23,24,25,26,27,28,29,30,31,32,33,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (45,78) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (42,43,86) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (42,43,44,45,54,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_rai

##### Identifying common columns between all


In [15]:
cols = set.intersection(*[set(x) for x in cols])


In [16]:
cols

{'AA1',
 'AA2',
 'AA3',
 'AT1',
 'AT2',
 'AT3',
 'AT4',
 'AW1',
 'AW2',
 'CALL_SIGN',
 'CIG',
 'DATE',
 'DEW',
 'ELEVATION',
 'EQD',
 'GA1',
 'GA2',
 'GA3',
 'GD1',
 'GD2',
 'GD3',
 'GF1',
 'KA1',
 'KA2',
 'LATITUDE',
 'LONGITUDE',
 'MA1',
 'MD1',
 'MW1',
 'NAME',
 'OC1',
 'OD1',
 'QUALITY_CONTROL',
 'REM',
 'REPORT_TYPE',
 'SLP',
 'SOURCE',
 'STATION',
 'TMP',
 'VIS',
 'WND'}

In [17]:
li = []
for file in files:
    dfWeather = pd.read_csv(file,index_col= None, header = 0, usecols=cols)
    li.append(dfWeather)
dfWeather = pd.concat(li, axis = 0, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
dfWeather.DATE =  dfWeather.DATE.str.replace('T',' ')

In [19]:
dfWeather.DATE

0         2010-01-01 00:51:00
1         2010-01-01 01:51:00
2         2010-01-01 02:37:00
3         2010-01-01 02:51:00
4         2010-01-01 03:51:00
                 ...         
132890    2021-04-30 21:51:00
132891    2021-05-01 04:51:00
132892    2021-05-01 04:59:00
132893    2021-05-01 05:51:00
132894    2021-05-01 06:51:00
Name: DATE, Length: 132895, dtype: object

### Below code is for cleaning the data set of missing values 

In [20]:
dfWeather = dfWeather.loc[:,['DATE','WND', 'TMP', 'DEW']]

In [21]:
dfWeather = dfWeather.sort_values(by = ['DATE'])

In [22]:
dfWeather

,DATE,WND,TMP,DEW
0,2010-01-01 00:51:00,"040,1,N,0015,1","+0010,1","-0010,1"
1,2010-01-01 01:51:00,"999,9,C,0000,1","+0010,1","-0010,1"
2,2010-01-01 02:37:00,"999,9,C,0000,1","+0010,1","-0010,1"
3,2010-01-01 02:51:00,"999,9,C,0000,1","+0010,1","-0010,1"
4,2010-01-01 03:51:00,"060,1,N,0026,1","+0010,1","+0000,1"
...,...,...,...,...
132890,2021-04-30 21:51:00,"290,5,N,0057,5","+0183,5","-0033,5"
132891,2021-05-01 04:51:00,"310,5,V,0067,5","+0078,6","-0044,5"
132892,2021-05-01 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
132893,2021-05-01 05:51:00,"999,9,V,0031,5","+0072,5","-0039,5"


In [23]:
dfWeather.loc[:,'TMP'].apply(lambda x: len(x)).sort_values()

0         7
88604     7
88603     7
88602     7
88601     7
         ..
44290     7
44289     7
44288     7
44318     7
132894    7
Name: TMP, Length: 132895, dtype: int64

In [24]:
dfWeather[dfWeather.loc[:,'TMP'] == '+9999,9']

,DATE,WND,TMP,DEW
46,2010-01-02 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
73,2010-01-03 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
102,2010-01-04 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
177,2010-01-07 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
254,2010-01-09 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
...,...,...,...,...
132791,2021-04-27 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
132816,2021-04-28 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
132843,2021-04-29 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"
132870,2021-04-30 04:59:00,"999,9,9,9999,9","+9999,9","+9999,9"


In [25]:
dfWeather.tail(1).TMP

132894    +0067,5
Name: TMP, dtype: object

In [26]:
dfWeather.loc[:,'TMP'].sort_values()

10295    +0000,1
11815    +0000,1
11813    +0000,1
9768     +0000,1
9767     +0000,1
          ...   
68181    -0167,5
68177    -0172,5
68178    -0178,5
68180    -0178,5
68179    -0183,5
Name: TMP, Length: 132895, dtype: object

In [27]:
def tmpF(tmp):
    if tmp.startswith('-'):
        tmp = int(tmp[2:5])
        tmp = (-(tmp/10)*((9/5))+32)
    else:
        tmp = int(tmp[2:5])
        tmp = ((tmp/10)*(9/5)) + 32
    return int(tmp)

##### Applying the tmpF function to both air and dew tempratures, to conver it into fahrenheit and divding with 10 due to scaling factor mentioned in meta data

In [28]:
dfWeather.loc[:,'TMP'] = dfWeather.loc[:,'TMP'].replace(to_replace = '+9999,9', method = 'bfill').apply(tmpF)#iloc[842]

In [29]:
dfWeather.loc[:1,'TMP']

0    33
1    33
Name: TMP, dtype: int64

In [30]:
dfWeather.loc[:,'DEW'] =  dfWeather.loc[:,'DEW'].replace(to_replace = '+9999,9', method = 'bfill').apply(tmpF)

In [31]:
dfWeather.DEW.head(2)

0    30
1    30
Name: DEW, dtype: int64

In [32]:
dfWeather.WND

0         040,1,N,0015,1
1         999,9,C,0000,1
2         999,9,C,0000,1
3         999,9,C,0000,1
4         060,1,N,0026,1
               ...      
132890    290,5,N,0057,5
132891    310,5,V,0067,5
132892    999,9,9,9999,9
132893    999,9,V,0031,5
132894    310,5,N,0057,5
Name: WND, Length: 132895, dtype: object

##### Changing wind from m/s to mph & filliing missing values with back fill method 


In [33]:
dfWeather.loc[:,'WND'] =  dfWeather.loc[:,'WND'].apply(lambda wind:'0' if wind[8:12] == '9999' else wind).replace(to_replace = '0', method = 'bfill') 

In [34]:
dfWeather.loc[:,'WND'] = dfWeather.loc[:,'WND'].apply(lambda wind: int(wind[8:12])*2.237/10)#.describe()

In [35]:
dfWeather.describe()

,WND,TMP,DEW
count,132895.000000,132895.000000,132895.000000
mean,5.549410,54.517250,42.437955
std,3.806912,17.158326,19.125981
min,0.000000,0.000000,-18.000000
25%,3.355500,41.000000,28.000000
50%,5.816200,55.000000,44.000000
75%,8.053200,69.000000,59.000000
max,32.212800,102.000000,80.000000


In [36]:
#dfWeather.to_csv('weatherdata.csv', index= False)

##### Standardizing dates of Weather 

In [37]:
'2010-01-01 00:51:00'[11:]

'00:51:00'

In [38]:
dfWeather.DATE.apply(lambda x: x[11:])

0         00:51:00
1         01:51:00
2         02:37:00
3         02:51:00
4         03:51:00
            ...   
132890    21:51:00
132891    04:51:00
132892    04:59:00
132893    05:51:00
132894    06:51:00
Name: DATE, Length: 132895, dtype: object

In [39]:
dfWeather['TIME'] = dfWeather.DATE.apply(lambda x: x[11:])

In [40]:
dfWeather.TIME

0         00:51:00
1         01:51:00
2         02:37:00
3         02:51:00
4         03:51:00
            ...   
132890    21:51:00
132891    04:51:00
132892    04:59:00
132893    05:51:00
132894    06:51:00
Name: TIME, Length: 132895, dtype: object

In [41]:
def timeFunc(time):
    if time[:5] >= '23:30':
        time = '0'+str((int(time[:2]) - 23)) + ':00:00'
    elif time[3:] < '29:59':
        time = time[:3] + '00:00'
    else:
        if time[0] == '0':
            time = '0'+str(int(time[:2]) + 1) + ':00:00'
            if len(time) > 8:
                time = time[1:]
            else:
                pass
        else:
            time = str(int(time[:2]) + 1) + ':00:00'
    return time

In [42]:
dfWeather.TIME = dfWeather.TIME.apply(timeFunc)

In [43]:
dfWeather.DATE =  dfWeather.DATE.str[:11] + dfWeather.TIME

In [44]:
dfWeather.DATE

0         2010-01-01 01:00:00
1         2010-01-01 02:00:00
2         2010-01-01 03:00:00
3         2010-01-01 03:00:00
4         2010-01-01 04:00:00
                 ...         
132890    2021-04-30 22:00:00
132891    2021-05-01 05:00:00
132892    2021-05-01 05:00:00
132893    2021-05-01 06:00:00
132894    2021-05-01 07:00:00
Name: DATE, Length: 132895, dtype: object

In [45]:
dfWeather = dfWeather.drop_duplicates('DATE')

In [53]:
dfWeather = dfWeather.rename(columns= {'DATE':'DATES'})

In [54]:
dfWeather.DATES = pd.to_datetime(dfWeather.DATES)

In [55]:
dfWeather

,DATES,WND,TMP,DEW,TIME
0,2010-01-01 01:00:00,3.3555,33,30,01:00:00
1,2010-01-01 02:00:00,0.0000,33,30,02:00:00
2,2010-01-01 03:00:00,0.0000,33,30,03:00:00
4,2010-01-01 04:00:00,5.8162,33,32,04:00:00
5,2010-01-01 05:00:00,0.0000,33,30,05:00:00
...,...,...,...,...,...
132889,2021-04-30 21:00:00,12.7509,64,30,21:00:00
132890,2021-04-30 22:00:00,12.7509,64,26,22:00:00
132891,2021-05-01 05:00:00,14.9879,46,24,05:00:00
132893,2021-05-01 06:00:00,6.9347,44,24,06:00:00


In [56]:
dfWeather.set_index('DATES', drop= False)

,DATES,WND,TMP,DEW,TIME
DATES,,,,,
2010-01-01 01:00:00,2010-01-01 01:00:00,3.3555,33,30,01:00:00
2010-01-01 02:00:00,2010-01-01 02:00:00,0.0000,33,30,02:00:00
2010-01-01 03:00:00,2010-01-01 03:00:00,0.0000,33,30,03:00:00
2010-01-01 04:00:00,2010-01-01 04:00:00,5.8162,33,32,04:00:00
2010-01-01 05:00:00,2010-01-01 05:00:00,0.0000,33,30,05:00:00
...,...,...,...,...,...
2021-04-30 21:00:00,2021-04-30 21:00:00,12.7509,64,30,21:00:00
2021-04-30 22:00:00,2021-04-30 22:00:00,12.7509,64,26,22:00:00
2021-05-01 05:00:00,2021-05-01 05:00:00,14.9879,46,24,05:00:00


In [57]:
dfWeather = dfWeather.set_index('DATES', drop= False)#.rename(columns= {'DATE':'DATES'})

In [58]:
dfWeather.DATES

DATES
2010-01-01 01:00:00   2010-01-01 01:00:00
2010-01-01 02:00:00   2010-01-01 02:00:00
2010-01-01 03:00:00   2010-01-01 03:00:00
2010-01-01 04:00:00   2010-01-01 04:00:00
2010-01-01 05:00:00   2010-01-01 05:00:00
                              ...        
2021-04-30 21:00:00   2021-04-30 21:00:00
2021-04-30 22:00:00   2021-04-30 22:00:00
2021-05-01 05:00:00   2021-05-01 05:00:00
2021-05-01 06:00:00   2021-05-01 06:00:00
2021-05-01 07:00:00   2021-05-01 07:00:00
Name: DATES, Length: 98582, dtype: datetime64[ns]

In [59]:
start = datetime(2010, 1, 1)
end = datetime(2021, 5, 1)


In [60]:
dfDate = pd.DataFrame(pd.date_range(start, end, freq= '60min'), columns=['DATE'])

In [61]:
dfDate.DATE

0       2010-01-01 00:00:00
1       2010-01-01 01:00:00
2       2010-01-01 02:00:00
3       2010-01-01 03:00:00
4       2010-01-01 04:00:00
                ...        
99308   2021-04-30 20:00:00
99309   2021-04-30 21:00:00
99310   2021-04-30 22:00:00
99311   2021-04-30 23:00:00
99312   2021-05-01 00:00:00
Name: DATE, Length: 99313, dtype: datetime64[ns]

In [62]:
dfDate = dfDate.set_index('DATE')

In [63]:
dfWeather = dfDate.join(dfWeather)

In [64]:
dfWeather = dfWeather.drop('DATES', axis = 1)

In [65]:
dfWeather = dfWeather.fillna(method= 'ffill')

In [66]:
dfWeather.isna().sum()

WND     0
TMP     0
DEW     0
TIME    0
dtype: int64

In [67]:
dfLoadWeather = dfLoad.join(dfWeather)

In [68]:
dfLoadWeather = dfLoadWeather.iloc[:99313,:]

In [69]:
dfLoadWeather.isna().sum()

rtd_actual_load    12
WND                 0
TMP                 0
DEW                 0
TIME                0
dtype: int64

In [70]:
dfLoadWeather = dfLoadWeather.fillna(method ='ffill')

In [71]:
dfLoadWeather

,rtd_actual_load,WND,TMP,DEW,TIME
time_stamp,,,,,
2010-01-01 00:00:00,5385.227273,3.3555,39.0,30.0,00:00:00
2010-01-01 01:00:00,5192.575000,3.3555,33.0,30.0,01:00:00
2010-01-01 02:00:00,4978.225000,0.0000,33.0,30.0,02:00:00
2010-01-01 03:00:00,4784.608333,0.0000,33.0,30.0,03:00:00
2010-01-01 04:00:00,4696.741667,5.8162,33.0,32.0,04:00:00
...,...,...,...,...,...
2021-04-30 20:00:00,5007.288600,16.1064,66.0,39.0,20:00:00
2021-04-30 21:00:00,4878.820267,12.7509,64.0,30.0,21:00:00
2021-04-30 22:00:00,4644.580058,12.7509,64.0,26.0,22:00:00


In [72]:
dfLoadWeather = dfLoadWeather[~((dfLoadWeather.index.month == 2) & (dfLoadWeather.index.day == 29))]


In [73]:
dfLoadWeather.sample()

,rtd_actual_load,WND,TMP,DEW,TIME
time_stamp,,,,,
2019-01-23 16:00:00,6806.8,9.1717,35.0,15.0,16:00:00


In [75]:
dfHumididty = dfLoadWeather.copy()

In [76]:
temperature = [39.0] * units.degF
dew  = [30.0] * units.degF


In [77]:
dfHumididty['RH'] = dfHumididty.apply(lambda x: mpcalc.relative_humidity_from_dewpoint([x.TMP]*units.degF, [x.DEW]*units.degF) , axis = 1)

In [78]:
dfHumididty['RH'] = dfHumididty.RH.apply(lambda x: float(x[0]))

In [79]:
dfHumididty.describe()

,rtd_actual_load,WND,TMP,DEW,RH
count,99241.000000,99241.000000,99241.000000,99241.000000,99241.000000
mean,6002.830866,5.524586,55.187211,40.897039,0.613879
std,1287.556456,3.721243,17.697320,19.471385,0.195945
min,2861.616667,0.000000,0.000000,-18.000000,0.072259
25%,5049.141667,3.355500,41.000000,26.000000,0.464088
50%,5915.283333,5.816200,55.000000,42.000000,0.595932
75%,6639.175000,8.053200,71.000000,57.000000,0.770308
max,11452.216667,29.975800,102.000000,78.000000,1.596416


In [80]:
dfHumididty[dfHumididty.TMP < dfHumididty.DEW]

,rtd_actual_load,WND,TMP,DEW,TIME,RH
time_stamp,,,,,,
2019-03-01 17:00:00,6737.025,9.1717,15.0,26.0,17:00:00,1.596416


In [81]:
dfHumididty.loc['2019-03-01 17:00:00',:] = dfHumididty.loc['2019-03-01 18:00:00',:]

In [82]:
dfHumididty.corr()

,rtd_actual_load,WND,TMP,DEW,RH
rtd_actual_load,1.000000,-0.098028,0.430933,0.395721,0.047576
WND,-0.098028,1.000000,-0.329330,-0.350077,-0.144384
TMP,0.430933,-0.329330,1.000000,0.888650,0.093020
DEW,0.395721,-0.350077,0.888650,1.000000,0.530335
RH,0.047576,-0.144384,0.093020,0.530335,1.000000


In [83]:
dfHumididty

,rtd_actual_load,WND,TMP,DEW,TIME,RH
time_stamp,,,,,,
2010-01-01 00:00:00,5385.227273,3.3555,39.0,30.0,00:00:00,0.698537
2010-01-01 01:00:00,5192.575000,3.3555,33.0,30.0,01:00:00,0.885837
2010-01-01 02:00:00,4978.225000,0.0000,33.0,30.0,02:00:00,0.885837
2010-01-01 03:00:00,4784.608333,0.0000,33.0,30.0,03:00:00,0.885837
2010-01-01 04:00:00,4696.741667,5.8162,33.0,32.0,04:00:00,0.960575
...,...,...,...,...,...,...
2021-04-30 20:00:00,5007.288600,16.1064,66.0,39.0,20:00:00,0.369996
2021-04-30 21:00:00,4878.820267,12.7509,64.0,30.0,21:00:00,0.277122
2021-04-30 22:00:00,4644.580058,12.7509,64.0,26.0,22:00:00,0.235147


In [85]:
dfHumididty.to_csv('FinalLoadWeather.csv')

In [84]:
pd.read_csv('FinalLoadWeather.csv')

,time_stamp,rtd_actual_load,WND,TMP,DEW,RH,CDD,weekday,month,season,day,hour,year,holidays
0,2010-01-01 00:00:00,5385.227273,3.3555,39.0,30.0,0.698537,0.0,4,1,winter,1,0,2010,True
1,2010-01-01 01:00:00,5192.575000,3.3555,33.0,30.0,0.885837,0.0,4,1,winter,1,1,2010,False
2,2010-01-01 02:00:00,4978.225000,0.0000,33.0,30.0,0.885837,0.0,4,1,winter,1,2,2010,False
3,2010-01-01 03:00:00,4784.608333,0.0000,33.0,30.0,0.885837,0.0,4,1,winter,1,3,2010,False
4,2010-01-01 04:00:00,4696.741667,5.8162,33.0,32.0,0.960575,0.0,4,1,winter,1,4,2010,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99235,2021-04-30 19:00:00,5089.433517,11.4087,55.0,46.0,0.716002,0.0,4,4,spring,30,19,2021,False
99236,2021-04-30 20:00:00,5007.288600,16.1064,66.0,39.0,0.369996,1.0,4,4,spring,30,20,2021,False
99237,2021-04-30 21:00:00,4878.820267,12.7509,64.0,30.0,0.277122,0.0,4,4,spring,30,21,2021,False
99238,2021-04-30 22:00:00,4644.580058,12.7509,64.0,26.0,0.235147,0.0,4,4,spring,30,22,2021,False
